In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

%matplotlib ipympl

In [ ]:
class SingleProbe:
    def __init__(self, probe_time, wait_time, transmission):
        self._probe_time = probe_time
        self._wait_time = wait_time
        self._transmission = transmission

    def time_series(self, sample_rate):
        probe_data = np.ones(int(self._probe_time * sample_rate)) * self._transmission
        wait_data = np.zeros(int(self._wait_time * sample_rate))
        return np.append(probe_data, wait_data)

    def probe_times(self):
        return np.array([0, self._probe_time])

    def duration(self):
        return self._probe_time + self._wait_time

class MultipleProbes:
    def __init__(self, probes_single, pre_wait_time, post_wait_time):
        self._probes_single = probes_single
        self._pre_wait_time = pre_wait_time
        self._post_wait_time = post_wait_time

    def time_series(self, sample_rate):
        data = np.zeros(int(self._pre_wait_time * sample_rate))
        for probe in self._probes_single:
            data = np.append(data, probe.time_series(sample_rate))
        data = np.append(data, np.zeros(int(self._post_wait_time * sample_rate)))
        return data

    def probe_times(self):
        time_offset = self._pre_wait_time
        times = []
        for probe in self._probes_single:
            times.append(time_offset + probe.probe_times)
            time_offset += probe.duration
        return np.array(times)

    def duration(self):
        total_time = self._pre_wait_time + self._post_wait_time
        for probe in self._probes_single:
            total_time += probe.duration
        return total_time

class DetectSequence:
    def __init__(self, multiple_probes, repeats, pre_wait_time, post_wait_time):
        self._multiple_probes = multiple_probes
        self._repeats = repeats
        self._pre_wait_time = pre_wait_time
        self._post_wait_time = post_wait_time

    def time_series(self, sample_rate):
        data = np.zeros(int(self._pre_wait_time * sample_rate))
        data = np.append(data, np.tile(self._multiple_probes.time_series, self._repeats))
        data = np.append(data, np.zeros(int(self._post_wait_time * sample_rate)))
        return data

    def probe_times(self):
        time_offset = self._pre_wait_time
        probe_time = self._multiple_probes.duration()
        times = []
        for kk in range(self._repeats):
            times.append([time_offset + kk for kk in probe.probe_times])
        for probe in self._probes_single:
            times.append([time_offset + kk for kk in probe.probe_times])
            time_offset += probe.duration
        return times

class TransmissionWithoutNoise:
    def __init__(self, detect_sequence, pre_wait_time, post_wait_time, sample_rate: int):
        self._detect_sequence = detect_sequence
        self._pre_wait_time = pre_wait_time
        self._post_wait_time = post_wait_time
        self._sample_rate = sample_rate
